In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import ipywidgets as widgets

# Fix animation limit for Jupyter Notebook (100MB)
mpl.rcParams['animation.embed_limit'] = 100  

# 1D Allen-Cahn Solver with Periodic Boundary Conditions

# Parameters
Nx = 200       # Number of grid points
dx = 1.0       # Grid spacing
dt = 0.01      # Time step
L = 1.0        # Relaxation parameter
A = 1.0        # Double-well potential coefficient
kappa_phi = 1.0  # Gradient energy coefficient
steps = 1000   # Number of time steps

# Initialize phi field with a sinusoidal wave
x = np.linspace(0, Nx * dx, Nx)
phi = 0.5 + 0.1 * np.sin(2 * np.pi * x / (Nx * dx))

# Function to compute Laplacian with periodic boundary conditions in 1D
def laplacian_1D_periodic(phi, dx):
    """Compute the Laplacian using finite differences with periodic boundaries in 1D."""
    return (np.roll(phi, 1) + np.roll(phi, -1) - 2 * phi) / dx**2

# Time evolution
phi_history = []
snapshot_interval = 50  # Store frames at intervals to optimize memory

for step in range(steps):
    lap_phi = laplacian_1D_periodic(phi, dx)
    
    # Compute the derivative of the double-well potential
    dF_dphi = 2 * A * phi * (1 - phi) * (1 - 2 * phi)
    
    # Update phi field
    phi += dt * L * (2 * kappa_phi * lap_phi - dF_dphi)
    
    # Store snapshots at intervals
    if step % snapshot_interval == 0:
        phi_history.append(phi.copy())

# Visualization of evolution
fig, ax = plt.subplots(figsize=(6, 4))
ax.set_xlim(0, Nx)
ax.set_ylim(-0.1, 1.1)
line, = ax.plot(x, phi_history[0], 'b-', lw=2)

def update_1D(frame):
    """Update function for animation"""
    line.set_ydata(phi_history[frame])
    ax.set_title(f'Time Step: {frame * snapshot_interval}')
    return [line]

# Create the animation using FuncAnimation
anim = FuncAnimation(fig, update_1D, frames=len(phi_history), interval=100, blit=False)

# --- Automatic Display Method ---
try:
    # Try JSHTML (interactive slider)
    display(HTML(anim.to_jshtml()))
except RuntimeError:
    # Fallback to HTML5 video if JSHTML is too large
    display(HTML(anim.to_html5_video()))

# --- Add Interactive Buttons for Play, Loop, Stop ---
def run_loop(_):
    anim.event_source.start()
    
def run_once(_):
    anim.event_source.stop()
    anim.frame_seq = anim.new_frame_seq()
    anim.event_source.start()
    
def stop_animation(_):
    anim.event_source.stop()

# Create buttons
button_loop = widgets.Button(description="Loop")
button_once = widgets.Button(description="Once")
button_stop = widgets.Button(description="Stop")

# Assign button actions
button_loop.on_click(run_loop)
button_once.on_click(run_once)
button_stop.on_click(stop_animation)

# Display buttons
display(widgets.HBox([button_loop, button_once, button_stop]))



# Gauss-Seidel Method

## Introduction
The **Gauss-Seidel method** is an iterative technique used to solve a system of linear equations of the form:

$$
Ax = b
$$

where:
- $A$ is an $n \times n$ matrix of coefficients,
- $x$ is the vector of unknowns,
- $b$ is the right-hand side vector.

It is an improvement over the **Jacobi method**, as it makes use of updated values during iteration, leading to faster convergence.

## Mathematical Formulation
For a given system:

$$
A x = b
$$

where $A$ is decomposed into:
- $D$: Diagonal elements of $A$,
- $L$: Lower triangular elements (excluding diagonal),
- $U$: Upper triangular elements (excluding diagonal),

we can rewrite $Ax = b$ as:

$$
(D + L)x = b - Ux
$$

which gives the iterative update formula:

$$
x_i^{(k+1)} = \frac{1}{A_{ii}} \left( b_i - \sum_{j=1}^{i-1} A_{ij} x_j^{(k+1)} - \sum_{j=i+1}^{n} A_{ij} x_j^{(k)} \right)
$$

where:
-  $x_i^{(k+1)}$ is the updated value at iteration $k+1$,
- $x_j^{(k+1)}$ values from the current iteration are used immediately when available.

## Steps to Solve Using Gauss-Seidel
1. **Initialize**: Start with an initial guess $x^{(0)}$.
2. **Iterate**: Compute each $x_i^{(k+1)}$ using the formula above.
3. **Check Convergence**: Stop if the difference between successive iterations is below a tolerance $\epsilon$
   (e.g., $||x^{(k+1)} - x^{(k)}|| < \epsilon$).
5. **Output Solution**: The final $x$ is the approximate solution.

## Advantages of Gauss-Seidel
- **Faster Convergence**: Uses updated values as soon as they are computed.
- **Less Memory Requirement**: Works in-place, modifying the solution vector directly.
- **Better Performance for Diagonally Dominant Matrices**: Ensures convergence when \( A \) is strictly diagonally dominant (i.e., $|A_{ii}| > \sum_{j \neq i} |A_{ij}|$).

## Example
Consider the system:

$$
\begin{bmatrix} 4 & -1 & 0 & 0 \\ -1 & 4 & -1 & 0 \\ 0 & -1 & 4 & -1 \\ 0 & 0 & -1 & 3 \end{bmatrix}
\begin{bmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{bmatrix}
=
\begin{bmatrix} 15 \\ 10 \\ 10 \\ 10 \end{bmatrix}
$$

Using Gauss-Seidel, we update $x_1, x_2, x_3, x_4$ iteratively.

If we start with $x^{(0)} = [0, 0, 0, 0]$, the first iteration computes:

$$
x_1^{(1)} = \frac{15 - (-1 \cdot 0) - (0 \cdot 0) - (0 \cdot 0)}{4} = 3.75
$$
$$
x_2^{(1)} = \frac{10 - (-1 \cdot 3.75) - (-1 \cdot 0) - (0 \cdot 0)}{4} = 3.4375
$$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Gauss-Seidel Method Implementation
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Solves Ax = b using the Gauss-Seidel iterative method.
    
    Parameters:
    A : numpy.ndarray
        Coefficient matrix (must be square and diagonally dominant).
    b : numpy.ndarray
        Right-hand side vector.
    x0 : numpy.ndarray, optional
        Initial guess (default is a zero vector).
    tol : float, optional
        Convergence tolerance (default is 1e-10).
    max_iter : int, optional
        Maximum number of iterations (default is 100).
    
    Returns:
    x : numpy.ndarray
        Solution vector.
    """
    n = len(b)
    if x0 is None:
        x = np.zeros_like(b, dtype=np.double)
    else:
        x = x0.astype(np.double)
    
    for k in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            sum1 = np.dot(A[i, :i], x[:i])
            sum2 = np.dot(A[i, i+1:], x_old[i+1:])
            x[i] = (b[i] - sum1 - sum2) / A[i, i]
        
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            break
    
    return x

# Example System
A = np.array([[4, -1, 0, 0],
              [-1, 4, -1, 0],
              [0, -1, 4, -1],
              [0, 0, -1, 3]], dtype=np.double)
b = np.array([15, 10, 10, 10], dtype=np.double)

# Solve using Gauss-Seidel
x_gs = gauss_seidel(A, b)
print("Solution using Gauss-Seidel:", x_gs)

# Compare with NumPy's solver
x_exact = np.linalg.solve(A, b)
print("Exact solution:", x_exact)

# Convergence Plot
errors = []
x = np.zeros_like(b, dtype=np.double)
for k in range(20):
    x_old = x.copy()
    for i in range(len(b)):
        sum1 = np.dot(A[i, :i], x[:i])
        sum2 = np.dot(A[i, i+1:], x_old[i+1:])
        x[i] = (b[i] - sum1 - sum2) / A[i, i]
    errors.append(np.linalg.norm(x - x_exact, ord=np.inf))

plt.figure(figsize=(8,5))
plt.semilogy(errors, marker='o', label='Gauss-Seidel Error')
plt.xlabel('Iteration')
plt.ylabel('Error (Infinity Norm)')
plt.title('Convergence of Gauss-Seidel Method')
plt.legend()
plt.grid()
plt.show()


## Semi-Implicit Finite Difference Implementation of Allen-Cahn with Gauss-Seidel

We treat **only the linear terms semi-implicitly**, while keeping the **nonlinear terms explicit**.

**Semi-Implicit Discretization:**
$$
\frac{\phi^{n+1} - \phi^n}{\Delta t} = -L \left( 2 A \phi^n (1 - \phi^n) (1 - 2\phi^n) - 2\kappa_\phi \nabla^2 \phi^{n+1} \right)
$$




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import ipywidgets as widgets

# Fix animation embed limit (100MB)
mpl.rcParams['animation.embed_limit'] = 100  

# -------------------------
# Simulation Parameters
# -------------------------
Nx = 400         # Number of grid points
dx = 1.0         # Grid spacing
dt = 0.01       # Time step
L = 1.0          # Relaxation parameter
A = 1.0          # Strength of double-well potential
kappa_phi = 1.0  # Strength of gradient energy term
steps = 10000    # Number of time steps
GS_iterations = 10  # Number of fixed-point iterations per time step
snapshot_interval = 50  # Save every snapshot_interval steps

# -------------------------
# Initial Condition: Sinusoidal Wave
# -------------------------
# Using a sinusoidal perturbation that spans multiple wavelengths.
x = np.linspace(0, Nx*dx, Nx)
# Here we use a six-wavelength sinusoid; adjust multiplier as needed.
phi = 0.5 + 0.2 * np.sin(4 * 2 * np.pi * x / (Nx*dx))

# -------------------------
# Function: Laplacian with Periodic Boundary Conditions
# -------------------------
def laplacian_1D_periodic(phi, dx):
    """
    Compute the Laplacian using central differences with periodic boundaries.
    For a grid point i:
      (phi[i+1] + phi[i-1] - 2*phi[i]) / dx^2
    with phi[-1] = phi[N-1] and phi[N] = phi[0].
    """
    return (np.roll(phi, -1) + np.roll(phi, 1) - 2 * phi) / dx**2

# -------------------------
# Semi-Implicit Time Evolution with Fixed-Point (Gauss-Seidel) Iterations
# -------------------------
# We use the following update:
#    phi_new = phi_old - dt*L*f(phi_old) + 2*dt*L*kappa_phi * laplacian(phi_new)
# where f(phi) = 2*A*phi*(1-phi)*(1-2phi) is computed explicitly using phi_old.
phi_history = []

for step in range(steps):
    phi_old = phi.copy()  # Save current time level
    # Compute the explicit nonlinear term f(phi_old)
    f_phi = 2 * A * phi_old * (1 - phi_old) * (1 - 2 * phi_old)
    
    # Fixed-point iteration for the implicit Laplacian term:
    # Initialize the new value with the old value (or use current phi)
    phi_new = phi_old.copy()
    for _ in range(GS_iterations):
        lap_phi = laplacian_1D_periodic(phi_new, dx)
        # Update: note that the Laplacian term is computed on the new guess.
        phi_new = phi_old - dt * L * f_phi + 2 * dt * L * kappa_phi * lap_phi
    
    # Set the new time step
    phi = phi_new.copy()
    
    # Save snapshot every snapshot_interval steps
    if step % snapshot_interval == 0:
        phi_history.append(phi.copy())

# -------------------------
# Visualization: Create Animation
# -------------------------
fig, ax = plt.subplots(figsize=(8, 4))
ax.set_xlim(0, Nx)
ax.set_ylim(-0.1, 1.1)
line, = ax.plot(x, phi_history[0], 'b-', lw=2)
ax.set_xlabel('Position')

def update_1D(frame):
    line.set_ydata(phi_history[frame])
    ax.set_title(f'Time Step: {frame * snapshot_interval}')
    return [line]

anim = FuncAnimation(fig, update_1D, frames=len(phi_history), interval=100, blit=False)

# -------------------------
# Automatic Display Method
# -------------------------
try:
    display(HTML(anim.to_jshtml()))
except RuntimeError:
    display(HTML(anim.to_html5_video()))

# -------------------------
# Interactive Buttons: Loop, Once, Stop
# -------------------------
def run_loop(_):
    anim.event_source.start()
    
def run_once(_):
    anim.event_source.stop()
    anim.frame_seq = anim.new_frame_seq()
    anim.event_source.start()
    
def stop_animation(_):
    anim.event_source.stop()

button_loop = widgets.Button(description="Loop")
button_once = widgets.Button(description="Once")
button_stop = widgets.Button(description="Stop")

button_loop.on_click(run_loop)
button_once.on_click(run_once)
button_stop.on_click(stop_animation)

display(widgets.HBox([button_loop, button_once, button_stop]))


In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import ipywidgets as widgets
import os

# Fix animation limit to 100MB
mpl.rcParams['animation.embed_limit'] = 100  

# Parameters
Nx = 200       # Number of grid points
dx = 1.0       # Grid spacing
dt_AC = 0.001  # Time step for Allen-Cahn
dt_CH = 0.01   # Larger time step for Cahn-Hilliard
L = 1.0        # Relaxation parameter
A = 1.0        # Double-well potential coefficient
kappa_phi = 1.0  # Gradient energy coefficient
kappa_CH = 1.0   # Gradient energy coefficient for CH
steps_AC = 20000   # Number of time steps for Allen-Cahn
steps_CH = 40000  # Increased steps for Cahn-Hilliard

# Define the domain
x = np.linspace(0, Nx * dx, Nx)

# Initial conditions: Four wavelengths
phi_AC = 0.5 + 0.1 * np.sin(4 * 2 * np.pi * x / (Nx * dx))  # Allen-Cahn
phi_CH = 0.5 + 0.2 * np.sin(4 * 2 * np.pi * x / (Nx * dx))  # Larger perturbation for CH

# Function to compute Laplacian in 1D with periodic boundaries
def laplacian_1D(phi, dx):
    return (np.roll(phi, 1) + np.roll(phi, -1) - 2 * phi) / dx**2

# Preallocate lists for visualization snapshots
snapshot_interval = 50
phi_AC_history = []
phi_CH_history = []

# Time evolution for Allen-Cahn
for step in range(steps_AC):
    lap_phi = laplacian_1D(phi_AC, dx)
    dF_dphi = 2 * A * phi_AC * (1 - phi_AC) * (1 - 2 * phi_AC)
    phi_AC += dt_AC * L * (2 * kappa_phi * lap_phi - dF_dphi)
    
    if step % snapshot_interval == 0:
        phi_AC_history.append(phi_AC.copy())

# Time evolution for Cahn-Hilliard (with kappa_CH = 1)
for step in range(steps_CH):
    lap_phi = laplacian_1D(phi_CH, dx)
    mu = 2 * A * phi_CH * (1 - phi_CH) * (1 - 2 * phi_CH) - 2 * kappa_CH * lap_phi
    lap_mu = laplacian_1D(mu, dx)
    phi_CH += dt_CH * L * lap_mu  # Using larger dt for CH
    
    if step % snapshot_interval == 0:
        phi_CH_history.append(phi_CH.copy())

# Ensure both animations have the same number of frames
num_frames = min(len(phi_AC_history), len(phi_CH_history))  

# Visualization of comparison between Allen-Cahn and Cahn-Hilliard
fig, ax = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
ax[0].set_xlim(0, Nx)
ax[0].set_ylim(-0.1, 1.1)
ax[1].set_ylim(-0.1, 1.1)

line_AC, = ax[0].plot(x, phi_AC_history[0], 'b-', lw=2, label="Allen-Cahn")
line_CH, = ax[1].plot(x, phi_CH_history[0], 'r-', lw=2, label="Cahn-Hilliard")

ax[0].set_title('Allen-Cahn Evolution (Four Wavelengths)')
ax[1].set_title('Cahn-Hilliard Evolution (Four Wavelengths, kappa_CH = 1)')
ax[1].set_xlabel('Position')

ax[0].legend()
ax[1].legend()

def update_comparison(frame):
    line_AC.set_ydata(phi_AC_history[frame])
    line_CH.set_ydata(phi_CH_history[frame])
    ax[0].set_title(f'Allen-Cahn Evolution - Time Step: {frame * snapshot_interval}')
    ax[1].set_title(f'Cahn-Hilliard Evolution (kappa_CH = 1) - Time Step: {frame * snapshot_interval}')
    return [line_AC, line_CH]

# Create the animation using FuncAnimation
animation_ref = FuncAnimation(fig, update_comparison, frames=num_frames, interval=200, blit=False)

# --- Choose Animation Display Mode ---
try:
    # Try using to_jshtml() for interactive animation
    print("Displaying animation using to_jshtml() (interactive slider).")
    display(HTML(animation_ref.to_jshtml()))
except RuntimeError:
    # If JSHTML fails due to large file size, use HTML5 video instead
    print("JSHTML failed. Displaying animation as HTML5 video (smaller file size).")
    display(HTML(animation_ref.to_html5_video()))

# --- Save Animation to MP4 File ---
mp4_filename = "AllenCahn_CahnHilliard.mp4"
try:
    animation_ref.save(mp4_filename, writer="ffmpeg", fps=15)
    print(f"Animation saved as {mp4_filename}")
except Exception as e:
    print(f"Could not save animation to MP4: {e}")

# --- Add Interactive Buttons for Loop, Once, Stop ---
def run_loop(_):
    animation_ref.event_source.start()
    
def run_once(_):
    animation_ref.event_source.stop()
    animation_ref.frame_seq = animation_ref.new_frame_seq()
    animation_ref.event_source.start()
    
def stop_animation(_):
    animation_ref.event_source.stop()

# Create buttons
button_loop = widgets.Button(description="Loop")
button_once = widgets.Button(description="Once")
button_stop = widgets.Button(description="Stop")

# Assign button actions
button_loop.on_click(run_loop)
button_once.on_click(run_once)
button_stop.on_click(stop_animation)

# Display buttons
display(widgets.HBox([button_loop, button_once, button_stop]))

# --- Display MP4 Download Link ---
if os.path.exists(mp4_filename):
    print(f"Download animation: {mp4_filename}")
    display(HTML(f'<a href="{mp4_filename}" download>Click here to download animation</a>'))
